In [ ]:
!pip install geopandas

In [ ]:
!pip install folium

### 어린이집 데이터 불러오기

In [24]:
# [1] 필요한 라이브러리 불러오기
import pandas as pd

# [2] CSV 파일 경로 지정
file_path = r"Data\서울시 어린이집 정보(표준 데이터).csv"

# [3] CSV 파일 불러오기 (인코딩 지정!)
df = pd.read_csv(file_path, encoding='cp949')



### 광진구 운영중인 어린이집

In [ ]:
# 필터링 조건 적용
df_filtered = df[
    (df['시군구명'] == '광진구') &
    (df['운영현황'] == '정상') &
    (df['시설 위도(좌표값)'].notnull()) &
    (df['시설 경도(좌표값)'].notnull())
]

# 저장할 컬럼만 추출
columns_to_save = [
    '어린이집명', '어린이집유형', '운영현황', '제공서비스',
    '시설 위도(좌표값)', '시설 경도(좌표값)', '상세주소',
    '전화번호', '홈페이지주소',
    '정원', '현원', '보육교직원수', '보육실수', '보육실 면적',
    '놀이터수', 'CCTV총설치수', '통학차량운영여부'
]

df_result = df_filtered[columns_to_save]

,어린이집명,어린이집유형,운영현황,제공서비스,시설 위도(좌표값),시설 경도(좌표값),상세주소,전화번호,홈페이지주소,정원,현원,보육교직원수,보육실수,보육실 면적,놀이터수,CCTV총설치수,통학차량운영여부
2,구립 자양이스트폴어린이집,국공립,정상,장애아통합,37.534656,127.088955,서울특별시 광진구 뚝섬로 717 구립 자양이스트폴어린이집,02-450-7547,NaN,98,0,2,9,793.0,1,0,NaN
13,구립 해달별어린이집,국공립,정상,일반,37.536761,127.090969,서울특별시 광진구 구의강변로3가길 39 701동 102호,02-455-7516,NaN,19,19,8,4,85.0,1,4,NaN
85,구립 새벽어린이집,국공립,정상,일반,37.555663,127.092531,서울특별시 광진구 영화사로 53 1층,02-458-5279,NaN,21,20,9,3,74.0,0,6,미운영
113,구립 자양롯데캐슬어린이집,국공립,정상,일반,37.537441,127.063639,서울특별시 광진구 뚝섬로 467,02-6082-0467,NaN,50,47,13,5,252.0,3,9,미운영
217,구립 광진그랜드파크어린이집,국공립,정상,일반,37.543706,127.081234,서울특별시 광진구 광나루로 458 어린이집,02-469-1956,NaN,57,37,11,5,176.0,1,15,미운영


In [33]:
df_active = df_result[
    (df['운영현황'] == '정상') &
    (df['현원'] > 0)
]
len(df_active)
df_active.to_csv("Data\광진구_실제운영중_어린이집.csv", index=False, encoding='utf-8-sig')


C:\Users\hsrhe\AppData\Local\Temp\ipykernel_26192\2225297131.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_active = df_result[


In [7]:
len(filtered_names)

298

In [16]:
import geopandas as gpd
import folium
import pandas as pd

# 시군구 Shapefile (정상 파일)
gdf = gpd.read_file(r"Data\sig_20230729\sig.shp", encoding='euc-kr')

# 읍면동 Shapefile (정상 파일)
gdf_emd = gpd.read_file(r"Data\emd_20230729 (1)\emd.shp", encoding='euc-kr')


## 어린이집 유형별 시각화 (운영중인곳만)

In [ ]:
import pandas as pd
import folium

# [1] 데이터 불러오기
file_path = r"Data\서울시 어린이집 정보(표준 데이터).csv"
df = pd.read_csv(file_path, encoding='cp949')

# [2] 운영 중인 어린이집만 필터링
df_active = df[df['운영현황'] == '정상']

# [3] 유형별 색상 매핑
type_color = {
    "국공립": "blue",
    "사회복지법인": "green",
    "민간": "orange",
    "가정": "purple",
    "직장": "pink",
    "협동": "gray",
    "기타": "lightgray"
}

# [4] 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# [5] 어린이집 마커 추가 (레이블 없이)
for _, row in df_active.iterrows():
    lat = row["시설 위도(좌표값)"]
    lon = row["시설 경도(좌표값)"]
    name = row["어린이집명"]
    type_ = str(row["어린이집유형"])
    phone = row.get("전화번호", "")
    color = type_color.get(type_, "gray")

    if pd.notnull(lat) and pd.notnull(lon):
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            color=color,
            fill=True,
            fill_opacity=0.7,
            tooltip=f"{name} ({type_})\n📞 {phone}"
        ).add_to(m)

# [6] 범례 추가 (HTML + CSS)
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 160px; height: auto; 
    border:2px solid grey; z-index:9999; font-size:12px;
    background-color: white; padding: 10px; border-radius: 8px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
<b>어린이집 유형</b><br>
<i style="background:blue; width:10px; height:10px; display:inline-block;"></i>&nbsp;국공립<br>
<i style="background:green; width:10px; height:10px; display:inline-block;"></i>&nbsp;사회복지법인<br>
<i style="background:orange; width:10px; height:10px; display:inline-block;"></i>&nbsp;민간<br>
<i style="background:purple; width:10px; height:10px; display:inline-block;"></i>&nbsp;가정<br>
<i style="background:pink; width:10px; height:10px; display:inline-block;"></i>&nbsp;직장<br>
<i style="background:gray; width:10px; height:10px; display:inline-block;"></i>&nbsp;협동<br>
<i style="background:lightgray; width:10px; height:10px; display:inline-block;"></i>&nbsp;기타
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# [7] 지도 출력
m


### 광진구 공원 불러오기

In [23]:
# [1] 필요한 라이브러리 불러오기
import pandas as pd

# [2] CSV 파일 경로 지정
park_file_path = r"Data\서울특별시_광진구_도시공원정보_20250310.csv"

# [3] CSV 파일 불러오기 (인코딩 지정!)
park_df = pd.read_csv(park_file_path, encoding='utf-8')

## 광진구 청크별로 나누기

In [42]:
# [1] 필요한 라이브러리 불러오기
import pandas as pd

# [2] CSV 파일 경로 지정
house_file_path = r"Data\gwangjin_geocoded_partial.csv"

# [3] CSV 파일 불러오기 (인코딩 지정!)
house_df = pd.read_csv(house_file_path, encoding='utf-8')

house_df.columns
house_df_filtered = house_df[house_df['세대수'] > 0]
len(house_df_filtered)


5220

In [34]:
import geopandas as gpd

# 시군구 Shapefile 불러오기 (예: sig.shp)
gdf_sig = gpd.read_file("Data/sig_20230729/sig.shp", encoding='euc-kr')

# 광진구만 필터링
gwangjin = gdf_sig[gdf_sig['SIG_KOR_NM'] == '광진구'].to_crs(epsg=5181)  # 미터 단위


ValueError: Cannot transform naive geometries.  Please set a crs on the object first.